# Greedy Agent function

* This agent takes advantage of board wrapping and goes for the closest food. It also avoids switching directions eg "WEST" to "EAST" which is not allowed.

###Add collision detection on other geese bodies


In [ ]:
def get_direction(food_row, player_row, food_column, player_column, forbidden_directions):
    direction = None
    row_difference = abs(food_row - player_row)
    col_difference = abs(food_column - player_column)

    other_row_difference = 7 - row_difference
    other_col_difference = 11 - col_difference

    if food_row > player_row:
        if row_difference > other_row_difference and "NORTH" not in forbidden_directions:
            direction = "NORTH"
        elif row_difference < other_row_difference and "SOUTH" not in forbidden_directions:
            direction = "SOUTH"
    if food_row < player_row:
        if row_difference < other_row_difference and "NORTH" not in forbidden_directions:
            direction = "NORTH"
        elif row_difference > other_row_difference and "SOUTH" not in forbidden_directions:
            direction = "SOUTH"

    if food_column > player_column:
        if col_difference < other_col_difference and "EAST" not in forbidden_directions:
            direction = "EAST"
        elif col_difference > other_col_difference  and "WEST" not in forbidden_directions:
            direction = "WEST"
    if food_column < player_column:
        if col_difference > other_col_difference and "EAST" not in forbidden_directions:
            direction = "EAST"
        elif col_difference < other_col_difference  and "WEST" not in forbidden_directions:
            direction = "WEST"
    
    return direction

In [ ]:
def get_closest_food(player_row, player_column, observation, configuration):
    foods = observation.food
    distances = []
    for i in range(len(foods)):
        food_row, food_column = row_col(foods[i], configuration.columns)
        total_distance = (food_row - player_row)**2 + (food_column - player_column)**2
        distances.append(total_distance)
    min_distance_index = distances.index(min(distances))
    closest_food = foods[min_distance_index]
    food_row, food_column = row_col(foods[min_distance_index], configuration.columns)
    return food_row, food_column

In [ ]:

def get_allgoose_directions(observation, configuration):

    number_of_geese = len(observation.geese)
    all_forbidden_directions = [] 
    all_allowed_directions = []
  #  print("New positions", new_positions)
   # print(observation.step)
    if observation.step == 0:
        for i in range(number_of_geese):
            previous_directions.append(None) 
            if len(new_positions) < 2:
                new_positions.append(None)
   # print("New positions after append", new_positions)
   # print(new_positions)
    banned_directions = {"EAST" : "WEST", "WEST" : "EAST", "NORTH" : "SOUTH", "SOUTH" : "NORTH"}
    directions_2 = {"NORTH" : Action.NORTH.name, "SOUTH" : Action.SOUTH.name, "EAST" : Action.EAST.name, "WEST" : Action.WEST.name}
    directions = {"NORTH" : [-1, 0], "SOUTH" : [1, 0], "EAST" : [0,1], "WEST" : [0,-1]}
    for i in range(number_of_geese):
        forbidden_directions = []
        allowed_directions = list(directions_2.values())
        previous_direction = previous_directions[i]
        if previous_direction is not None:
            banned_direction = banned_directions[previous_direction]
        else:
            banned_direction = None

        if banned_direction is not None:
            allowed_directions.remove(banned_direction)
            forbidden_directions.append(banned_direction)
            
        all_allowed_directions.append(allowed_directions)
        all_forbidden_directions.append(forbidden_directions)

        player_goose = observation.geese[i]
        player_head = player_goose[0]
        player_row, player_column = row_col(player_head, configuration.columns)
        food_row, food_column = get_closest_food(player_row, player_column, observation, configuration)
        direction = get_direction(food_row, player_row, food_column, player_column, forbidden_directions)
        #print("New positions before", new_positions)
        if direction is None:
            new_positions[i] = (player_row, player_column, direction)
        else:
            add_row = directions[direction][0]
            add_col = directions[direction][1]
            new_row = player_row + add_row
            new_column = player_column + add_col
            new_positions[i] = (new_row, new_column, direction)
        
    

        previous_direction = direction
        previous_directions[i] = previous_direction
    #print("new positions after", new_positions)
    return previous_directions, new_positions, all_forbidden_directions, all_allowed_directions
        

In [ ]:
def check_collision(observation, configuration, my_intended_direction, all_forbidden_directions, all_allowed_directions, my_index, my_new_row, my_new_col):
    my_goose = observation.geese[my_index]
    my_goose_head = my_goose[0]
    my_goose_row, my_goose_col = row_col(my_goose_head, configuration.columns)
    for i in range(len(my_goose)):
        row, col = row_col(my_goose[i], configuration.columns)
        if my_new_row == row and my_new_col == col:
            print("Will hit own body if move", my_intended_direction)
            all_forbidden_directions[my_index].append(my_intended_direction)
            if my_intended_direction in all_allowed_directions[my_index]:
                all_allowed_directions[my_index].remove(my_intended_direction)
    return all_forbidden_directions, all_allowed_directions

In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import random
global previous_directions
global new_positions
global directions
new_positions = []
previous_directions = []   
def agent(obs_dict, config_dict):
    
    observation = Observation(obs_dict)
    #print(observation.step)
    configuration = Configuration(config_dict)
    #print(configuration) 
    previous_directions, new_positions, all_forbidden_directions, all_allowed_directions = get_allgoose_directions(observation, configuration)
    my_index = observation.index
    #print("New positions",new_positions)
  #  print(all_forbidden_directions[my_index])
    (my_new_row, my_new_col, my_intended_direction) = new_positions[my_index]
    #print(new_positions)
    for i in range(len(new_positions)):
        (new_row, new_col, directions) = new_positions[i]
        if i != my_index and my_new_row == new_row and my_new_col == new_col:
            print("Will hit other snake if move ", my_intended_direction)
            all_forbidden_directions[my_index].append(my_intended_direction)
            if my_intended_direction in all_allowed_directions[my_index]:
                all_allowed_directions[my_index].remove(my_intended_direction)
    
    while my_intended_direction in all_forbidden_directions[my_index]:
        if len(all_allowed_directions[my_index]) > 0:
            my_intended_direction = random.choice(all_allowed_directions[my_index])
        else:
            my_intended_direction = random.choice(list(directions_2.values()))
        all_forbiden_directions, all_allowed_directions = check_collision(observation, configuration, my_intended_direction, all_forbidden_directions, all_allowed_directions, my_index, my_new_row, my_new_col)
    previous_directions[my_index] = my_intended_direction
    return my_intended_direction  

In [ ]:
from kaggle_environments import evaluate, make, utils

# Setup a hungry_geese environment.
env = make("hungry_geese", debug = True)

# Greedy VS Random

In [ ]:
for i in range(1000):
    env.run([agent, "random"])
    #env.render(mode="ipython", width=500, height=450)

# Greedy VS Greedy

In [ ]:
def mean_reward(rewards):
    print(rewards)
    return sum(r[0] for r in rewards) / float(len(rewards))

# Run multiple episodes to estimate its performance.
print("Greedy vs Random Agent:", mean_reward(evaluate("hungry_geese", [agent, "random"], num_episodes=1000)))
print("Greedy vs Greedy:", mean_reward(evaluate("hungry_geese", [agent, agent], num_episodes=1000)))

## Write Submission File